In [ ]:
from mpcontribs.client import load_client
from mpcontribs.io.core.recdict import RecursiveDict
from mpcontribs.io.core.components.hdata import HierarchicalData
from mpcontribs.io.core.components.tdata import Table
from mpcontribs.io.core.components.gdata import Plot

In [ ]:
api_key = 'your-api-key-here' # from https://portal.mpcontribs.org/dashboard
client = load_client(api_key)

## project info & overview table
see https://portal.mpcontribs.org/carrier_transport

In [ ]:
project = 'carrier_transport'
info = client.projects.get_entry(pk=project, _fields=['_all']).result()
RecursiveDict(info)

In [ ]:
keys, subkeys = ['<mₑᶜᵒⁿᵈ>', '<S>', '<σ>', '<S²σ>'], ['n', 'p']
columns = ['.'.join([k, sk]) for k in keys for sk in subkeys]
data = client.projects.get_table(project=project, columns=columns).result()
columns = list(data['items'][0].keys()) # reset to include units
Table(data['items'], columns=columns, project=project, api_key=api_key)

## specific contribution

In [ ]:
cid = '5ac0a333d4f1443de26744ab'
contrib = client.contributions.get_entry(
    pk=cid, _fields=['_all']
).result()

### hierarchical data

In [ ]:
HierarchicalData(contrib['data']) # dictionary

### tabular data

In [ ]:
tid = contrib['tables'][0]['id']
table = client.tables.get_entry(pk=tid, _fields=['_all']).result()
Table.from_dict(table) # Pandas DataFrame format

In [ ]:
Plot.from_dict(table) # plotly graph

## query by materials identifiers

In [ ]:
identifiers = ['mp-2715', 'mp-988', 'mp-9899']
contributions = client.contributions.get_entries(
    identifier__in=identifiers,
    _fields=['identifier', 'data.formula', 'data.<S>']
).result()['data']
for contrib in contributions:
    print(contrib['identifier'])
    HierarchicalData(contrib['data'])._ipython_display_()

## query by values

In [ ]:
# see https://portal.mpcontribs.org/<project> and
# https://api.mpcontribs.org/#/contributions/get_entries

per_page = 20 # 200 is max retrievable contributions per page
fields = ['<S>', '<σ>', '<S²σ>'] # which data fields to retrieve
mask = [f'data.{field}' for field in fields]
mask += ['data.formula', 'identifier']
filters = {
    'data__formula__contains': 'Li3',
#     'data__<σ>__p__lt': 2e15,
#     'data__<σ>__n__lt': 2e15
}

contribs = client.contributions.get_entries(
    project=project, _fields=mask, _limit=per_page, **filters
).result()['data'] # -> see pagination

print('found', len(contribs), 'materials')
contribs[0]

## pagination

In [ ]:
# TODO outdated
# only run this once you've optimized masks and filters to what you actually need (see above)
all_contribs = []
while 1:
    page = int(len(all_contribs) / per_page) + 1
    contribs = client.contributions.get_entries(
        projects=[project], filters=filters, mask=mask, per_page=per_page, page=page
    ).result()
    all_contribs.extend(contribs)
    if not contribs or len(contribs) < per_page:
        break

print('found', len(all_contribs), 'materials')
all_contribs[-1]

## save data for download
Save this notebook after modifying and executing its code. Then click on the Jupyter logo at the top to get back to the file browser and select files for download. Note that Binder sessions are [ephemeral](https://mybinder.readthedocs.io/en/latest/faq.html#how-long-will-my-binder-session-last), so make sure to download this notebook (and your data) before closing the session.

In [ ]:
import json
with open('my_data.json', 'w') as f:
    json.dump(all_contribs, f)